In [ ]:
import praw
import pandas as pd

# Initialize the Reddit client
reddit = praw.Reddit(
    client_id="4chqKwmOdorSuyav7KTIMg",      # Replace with your client ID
    client_secret="hI_HHldkP4pvfAsJ4xH9pWAkhGIn0g",  # Replace with your client secret
    user_agent="Sentiment-analysis",      
)

# Fetch posts from a subreddit with a keyword filter
def fetch_posts_about_keyword(subreddit_name, keywords, limit=1000):
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    
    for post in subreddit.hot(limit=limit):  # You can use new, top, etc.
        if any(keyword.lower() in (post.title + post.selftext).lower() for keyword in keywords):
            # Extract the post's comments
            post.comments.replace_more(limit=0)  # Replace 'more comments' placeholders
            comments = post.comments.list()  # Get all comments as a list
            
            for comment in comments:  # Create a row for each comment
                data.append({
                    'post_title': post.title,
                    'post_score': post.score,
                    'post_id': post.id,
                    'post_num_comments': post.num_comments,
                    'post_created': post.created,
                    'post_body': post.selftext,
                    'comment_id': comment.id,  # Add comment ID
                    'comment_body': comment.body, 
                    'comment_score': comment.score  
                })
    
    return data

# Example usage
subreddit_name = "stocks"  # Replace with your subreddit of interest
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]
data = fetch_posts_about_keyword(subreddit_name, keywords, limit=100)

# Save to a DataFrame
df = pd.DataFrame(data)
df.to_csv("Reddit_comments.csv", index=False)

print("Data saved to Reddit_comments.csv")


Data saved to Reddit_comments.csv
